In [297]:
import pandas as pd
import numpy as np
from scipy import stats


In [298]:
wsp_contract = pd.read_excel("../../Datasets/study-1-data.xlsx")



In [299]:
wsp_contract['Sales'] = wsp_contract[['Demand', 'Stock']].min(axis=1)

In [300]:
wsp_contract['Leftovers'] = wsp_contract['Stock'] - wsp_contract['Sales']

In [301]:
wsp_contract['Unmet_Demand'] = wsp_contract['Demand'] - wsp_contract['Sales']

In [302]:
wsp_contract.drop(columns=["Unnamed: 5"], inplace=True)

In [303]:
wsp_contract['Realized_Mfg_Profit'] = (wsp_contract['Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Stock']) 

In [304]:
wsp_contract['Realized_Retailer_Profit'] = (wsp_contract['Sales'] * 12) - (wsp_contract['Stock'] * wsp_contract['Wholesale p.'])

In [305]:
wsp_contract['Realized_Mfg_Profit_Share'] = wsp_contract['Realized_Mfg_Profit'] / (wsp_contract['Realized_Mfg_Profit'] + wsp_contract['Realized_Retailer_Profit']) 

In [306]:
wsp_contract['Optimal_Stock'] = 100*((12-wsp_contract['Wholesale p.'])/12) + 50
wsp_contract['Optimal_Stock'] = wsp_contract['Optimal_Stock'].astype(int)

In [307]:
wsp_contract['Under/Overstock_wrt_Optimal'] = wsp_contract['Stock'] - wsp_contract['Optimal_Stock']

In [308]:
potential_demand = np.arange(51, 151)
expected_sales = []

for index, row in wsp_contract.iterrows():
    cumulative_expected_sales = 0
    
    for demand_realization in potential_demand:
        sales = min(demand_realization, row['Stock'])

        cumulative_expected_sales += sales
        
        #if sales == row['Stock']:
         #   cumulative_expected_sales += (150 - demand_realization) * row['Stock']
          #  break
    
    # Calculate the average expected sales for this row and append it to the list
    expected_sales.append(cumulative_expected_sales / len(potential_demand))

wsp_contract['Expected Sales'] = np.array(expected_sales)

In [309]:
wsp_contract["Expected_Leftovers"] = wsp_contract["Optimal_Stock"] - wsp_contract["Expected Sales"]

In [310]:
wsp_contract['Expected Retailer Profit'] = (wsp_contract['Expected Sales'] * 12) - (wsp_contract['Wholesale p.'] * wsp_contract['Stock']) 

In [311]:
wsp_contract['Expected Mfg Profit'] = (wsp_contract['Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Stock'])

In [312]:
wsp_contract['Expected Mfg. Profit Share'] = wsp_contract['Expected Mfg Profit'] / (wsp_contract['Expected Mfg Profit'] + wsp_contract['Expected Retailer Profit']) #wsp_contract.apply(lambda row: row['Expected Mfg Profit'] / row['Total Expected Revenue'] if row['Total Expected Revenue'] != 0 else 0, axis=1)

In [313]:
potential_demand = np.arange(51, 151)
expected_sales = []

for index, row in wsp_contract.iterrows():
    cumulative_expected_sales = 0
    
    for demand_realization in potential_demand:
        sales = min(demand_realization, row['Optimal_Stock'])

        cumulative_expected_sales += sales
        
        #if sales == row['Stock']:
         #   cumulative_expected_sales += (150 - demand_realization) * row['Stock']
          #  break
    
    # Calculate the average expected sales for this row and append it to the list
    expected_sales.append(cumulative_expected_sales / len(potential_demand))

wsp_contract['Predicted Sales'] = np.array(expected_sales)

In [314]:
wsp_contract['Predicted Leftovers'] = wsp_contract['Optimal_Stock'] - wsp_contract['Predicted Sales']

In [315]:
wsp_contract['Predicted Retailer Profit'] = wsp_contract['Predicted Sales'] * 12 - (wsp_contract['Wholesale p.'] * wsp_contract['Optimal_Stock'])
wsp_contract['Predicted Mfg Profit'] = (wsp_contract['Optimal_Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Optimal_Stock'])

In [316]:
wsp_contract['Realized Retailer Profit'] = wsp_contract['Sales'] * 12 - (wsp_contract['Stock'] * wsp_contract['Wholesale p.'])

In [317]:
wsp_contract['Predicted Mfg. Profit Share'] = wsp_contract['Predicted Mfg Profit'] / (wsp_contract['Predicted Mfg Profit'] + wsp_contract['Predicted Retailer Profit'])

In [318]:
wsp_contract = wsp_contract.fillna(0)
wsp_contract.to_csv('./curr_data.csv', index = False)

In [319]:
grouped_data_retailer = wsp_contract.groupby('Player ID')


In [320]:
wsp_contract = pd.read_csv("../../Notebooks/experiment/curr_data.csv")

In [321]:
#Null Hypothesis (H0): The mean order quantity made by retailers is not significantly different from the optimal stock level.

#Alternative Hypothesis (H1): The mean order quantity made by retailers is significantly different from the optimal stock level.

#Reasoning: Retailers are expected to be able to adapt correctly to the changing wholesale price as they are rational agents.

wsp_contract['null'] = 0
from scipy.stats import wilcoxon

print(stats.shapiro(wsp_contract["Stock"]))
print(stats.shapiro(wsp_contract["Optimal_Stock"]))
_, p_value = wilcoxon((wsp_contract["Stock"]), wsp_contract['Optimal_Stock'])
print("p value:",p_value)
if p_value < 0.05:
    print("Reject the null hypothesis: The median absolute difference is significantly different from zero.")
else:
    print("Fail to reject the null hypothesis: No significant difference from zero in the median absolute difference.")

ShapiroResult(statistic=0.9213677048683167, pvalue=5.289216595482103e-21)
ShapiroResult(statistic=0.9109364151954651, pvalue=2.791107770045616e-22)
p value: 2.7778300779822927e-06
Reject the null hypothesis: The median absolute difference is significantly different from zero.


In [322]:
my = []
i = 0
for group_key, group_data in grouped_data_retailer:
    a=group_data.iloc[0:5]['Expected Retailer Profit'].mean()
    b=group_data.iloc[20:35]['Expected Retailer Profit'].mean()
    my.append([a, b])
    i+=1

my = pd.DataFrame(my)


#hypo 2
#H0: The retailers' expected profits in the first 5 rounds are not significantly different compared to later stages of the game.
#H1: There is statistically significant difference between the first 5 round and the later stages of the game.
#Resoning: We expect retailers to learn how to efficiently play the game and increase their expected profits in the later stages of the game


print(stats.shapiro(my[0]))
print(stats.shapiro(my[1]))


t_stat, p_value = stats.ttest_rel(my[0], my[1])
print("P value is", p_value, "which is higher than 0.05.\nMeaning that the null hypothesis is accepted.")


ShapiroResult(statistic=0.9673224687576294, pvalue=0.6491798758506775)
ShapiroResult(statistic=0.960820198059082, pvalue=0.5060364603996277)
P value is 0.4243008596576613 which is higher than 0.05.
Meaning that the null hypothesis is accepted.


In [323]:
#hypo 3: 

#H0: There is no statistically significant difference in the under/over ordering values of the best expected retailer profit scorers and the worst ones.
#meaning that the null hypothesis dictates that the retailers with highest expected retailer profits and lowest expected retailer profits have similar under/over ordering values
#Reasoning: We expect that the retailers whose expected profits are the highest among all should have come colser to the demand realization compared to the retailers with worst expected profits

liste = []
i = 0
for group_key, group_data in grouped_data_retailer:
    a=group_data['Expected Retailer Profit'].mean()
    liste.append([a, i])
    i+=1

liste = sorted(liste, key=lambda x: x[0], reverse=True)

liste 

[[386.0249999999999, 19],
 [347.746, 17],
 [326.69000000000005, 4],
 [313.69199999999995, 3],
 [303.936, 12],
 [302.41999999999996, 9],
 [289.32699999999994, 20],
 [265.582, 6],
 [256.01599999999996, 16],
 [251.417, 15],
 [249.76999999999992, 13],
 [247.94700000000003, 10],
 [246.148, 11],
 [244.06199999999995, 7],
 [228.01500000000001, 0],
 [227.025, 5],
 [223.231, 18],
 [221.73099999999994, 8],
 [191.922, 1],
 [184.99, 2],
 [184.80200000000005, 14],
 [170.87299999999996, 21]]

In [324]:
first_indexes = [21, 1, 16, 18, 2]
last_indexes = [17, 0, 9, 5, 14]
df_best = pd.DataFrame()
df_worst = pd.DataFrame()

i = 0
for group_key, group_data in grouped_data_retailer:
    if i in first_indexes:
        df_best = pd.concat([df_best, group_data])
    elif i in last_indexes:
        df_worst = pd.concat([df_worst, group_data])
    i+=1
from scipy.stats import wilcoxon
print(stats.shapiro(abs(df_best['Under/Overstock_wrt_Optimal'])))
print(stats.shapiro(abs(df_worst['Under/Overstock_wrt_Optimal'])))


res = wilcoxon(abs(df_best['Under/Overstock_wrt_Optimal']) ,abs(df_worst['Under/Overstock_wrt_Optimal'])) 

print(res)
print("Null hypothesis is rejected")

ShapiroResult(statistic=0.926517903804779, pvalue=1.800900228943192e-08)
ShapiroResult(statistic=0.8850858211517334, pvalue=3.114175931018259e-11)
WilcoxonResult(statistic=7102.0, pvalue=0.0018774414138789553)
Null hypothesis is rejected


In [325]:

#hypothesis 4
#Retailers could send a signal to manufacturers by either ordering 0 stock or 51 stock in which case the profits are 0 or the lowest possible value other than zero
#Retailers do this when they want to express that the manufacturers' wholesale price offers are not acceptable by the retailers. This can affect the next rounds
#H0: The wholesale price offers of manufacturers are not significantly affected by the signals a retailer sends them the in the previous round.



main_df = pd.DataFrame()
for group_key, group_data in grouped_data_retailer:

    df_a = group_data[((group_data['Stock'] == 0 ) | (group_data['Stock'] == 51 )) & (group_data.index != group_data.index[-1]) ]
    indices = df_a.index + 1
    df_b =wsp_contract.iloc[indices]
    df_a = df_a.reset_index(drop =True)
    df_b = df_b.reset_index(drop =True)
    df_c = pd.concat([df_a['Wholesale p.'], df_b['Wholesale p.']], axis = 1)
    main_df = pd.concat([main_df, df_c],)

main_df.columns = ['a', 'b']

print(stats.shapiro(main_df['a']))
print(stats.shapiro(main_df['b']))


_, p_value = wilcoxon(main_df['a'], main_df['b'])
print(p_value ,"reject h0: signals do not affect the next turn's wholesale price offer, hypo 4")
print("We see in the end that when there is a signal by the retailer, the next round's wholesale price offer is affected by this decision")

ShapiroResult(statistic=0.890314519405365, pvalue=2.6951266107744232e-08)
ShapiroResult(statistic=0.9140936136245728, pvalue=5.084256144982646e-07)
1.3175845443509359e-06 reject h0: signals do not affect the next turn's wholesale price offer, hypo 4
We see in the end that when there is a signal by the retailer, the next round's wholesale price offer is affected by this decision


In [326]:
#hypo 5
# H0: There is no difference in means between the predicted Mfg. Profit Share and Expected Mfg Profit.

#reasoning: Predicted manufacturer profit relies on the retailer making optimal decisions. If they are making optimal decisions, then the difference of means of these datasets whould be 0.

#Result: P value is less than 0.05, rejecting the null hypothesis. Retailers do not make optimal decisions.
from scipy import stats

#Since p values of the shapiro tests are less than 0.05, these sets reject normality, so wilcoxon needs to be used instead of something like t-test.
print(stats.shapiro(wsp_contract['Predicted Mfg Profit']))
print(stats.shapiro(wsp_contract['Expected Mfg Profit']))


_, p_val = stats.wilcoxon(wsp_contract['Predicted Mfg Profit'], wsp_contract['Expected Mfg Profit'])

print(p_val, "\n", "Reject H0")

ShapiroResult(statistic=0.8194721341133118, pvalue=2.708517095198587e-30)
ShapiroResult(statistic=0.9152687788009644, pvalue=9.163949496737127e-22)
0.0008860172293888648 
 Reject H0
